In [1]:
import warnings
from importlib import reload
from functools import partial, lru_cache
import itertools

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets, svm, metrics
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm, trange
from joblib import delayed
from modAL import batch
from art.metrics import empirical_robustness
from art.attacks.evasion import *
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from tabulate import tabulate
from art.estimators.classification.scikitlearn import ScikitlearnSVC
from sklearn.metrics.pairwise import paired_distances, euclidean_distances
import scipy
from tvregdiff.tvregdiff import TVRegDiff

from ipynb.fs.defs import Bias
from ipynb.fs.defs.Datasets import generateData_twoPills_2D, generateData_twoPills_noNoise_2D, plot_dataset_2D

import libactive
import libadversarial
import libstop
from libactive import MyActiveLearner, active_split
from libadversarial import adversarial, uncertainty, random_batch, uncertainty_stop
from libutil import ProgressParallel
from libdatasets import *
import librun
from librun import run

Using sklearn


In [7]:
import libdatasets; reload(libdatasets); from libdatasets import newsgroups

In [37]:
matrix = {
    # Dataset fetchers should cache if possible
    # Lambda wrapper required for function to be pickleable (sent to other threads via joblib)
    "datasets": [
        # Text classification
        
        # https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.31.6090&rep=rep1&type=pdf
        ("newsgroups_faith", lambda: lru_cache()(newsgroups)(('alt.atheism', 'soc.religion.christian'))),
        ("newsgroups_graphics", lambda: lru_cache()(newsgroups)(('comp.graphics', 'comp.windows.x'))),
        ("newsgroups_hardware", lambda: lru_cache()(newsgroups)(('comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware'))),
        #("newsgroups_sports_crypto", lambda: lru_cache()(newsgroups)(('rec.sports.baseball', 'sci.cryptography'))),
    
        ("rcv1", lambda: lru_cache()(rcv1)()),
        ("webkb", lambda: lru_cache()(webkb)()),
        ("spamassassin", lambda: lru_cache()(spamassassin)()),
        
        # Image classification
        
        ("cifar10", lambda: lru_cache()(cifar10)()),
        ("quickdraw", lambda: lru_cache()(quickdraw)()),
        
        # General
        ("banknote", lambda: lru_cache()(banknote)()),
        ("shuttle", lambda: lru_cache()(shuttle)()),
        ("sonar", lambda: lru_cache()(sonar)()), # 208 instances
        
        # Bio
        ("abalone", lambda: lru_cache()(abalone)()),
        ("splice", lambda: lru_cache()(splice)()),
        
        # Medical
        ("haberman", lambda: lru_cache()(haberman)()),
        ("cardio", lambda: lru_cache()(cardio)()),
        ("skin", lambda: lru_cache()(skin)()),
        
    ],
    "dataset_mutators": {
        "none": (lambda *x, **kwargs: x),
    },
    "methods": [
        ("uncertainty", partial(uncertainty_stop, n_instances=10)),
    ],
    "models": [
        "svm-linear"
    ],
    "meta": {
        "dataset_size": 1000,
        "labelled_size": 0.01,
        "n_runs": 10,
        "ret_classifiers": True,
        "ensure_y": True,
        "stop_info": True
    }
}

In [35]:
reload(libdatasets)

<module 'libdatasets' from 'C:\\Users\\Zac\\Programming\\python\\research\\libdatasets.py'>

In [38]:
libdatasets.dataset_summary([data[1] for data in matrix['datasets']])

2021-02-10 11:11:00,452 - py.warnings - WARNING - C:\Users\Zac\Programming\python\research\libdatasets.py:175: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  dataset = np.concatenate((dataset1, dataset2))

2021-02-10 11:11:01,068 - py.warnings - WARNING - C:\Users\Zac\Programming\python\research\libdatasets.py:176: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  y = dataset[:, -1] == 1

  Instances    Classes    Features  Most common class    Least common class
-----------  ---------  ----------  -------------------  --------------------
       1796          2      125145  15 56%               0 44%
      

In [14]:
results = librun.run(matrix, metrics=[accuracy_score, f1_score, roc_auc_score])

ValueError: With n_samples=0, test_size=0.5 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.